This notebook is intended to either train GPT2 from scratch using open-source implementations or to fine-tune a pre-existing model trained on GPT2 architecture.

Most recent version of this notebook was developed `by Josh Urban Davis (http://www.joshurbandavis.com, @joshurbandavis)`

Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

Most recent version of this notebook was developed by Josh Urban Davis (http://www.joshurbandavis.com, @joshurbandavis)

---



---



> Indented block



In [ ]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 4.40 MiB | 3.97 MiB/s, done.
Resolving deltas: 100% (168/168), done.


In [ ]:
cd gpt-2

/content/gpt-2


Install requirements

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 28.6MB/s 
     |████████████████████████████████| 604kB 44.1MB/s 
     |████████████████████████████████| 51kB 25.0MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=a0b50bba31d1b569c568a4b137cb502555c11cffd226746130dd303fa7e58a57
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533194 sha256=422776e6d00591ff0689f786679715d87ab6127eb9c1ddefe352599166416b81
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Download the model data

In [ ]:
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 717kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 56.1Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 957kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 62.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.29Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 40.2Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 51.5Mit/s]                                                       


In [ ]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 767kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 63.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 778kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:23, 60.7Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.96Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 67.4Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 59.7Mit/s]                                                       


encoding

In [ ]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [ ]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/


Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [ ]:
!wget https://www.gutenberg.org/files/98/98-0.txt

--2019-08-19 19:36:36--  https://www.gutenberg.org/files/98/98-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804335 (785K) [text/plain]
Saving to: ‘98-0.txt’

98-0.txt            100%[===================>] 785.48K  1.66MB/s    in 0.5s    

2019-08-19 19:36:36 (1.66 MB/s) - ‘98-0.txt’ saved [804335/804335]




Start training, add --model_name '345M' to use 345 model

In [ ]:
!PYTHONPATH=src ./train.py --dataset /content/obituary_text_corpus_GPT2_8.txt --model_name '345M'

W0904 03:39:00.654868 140429746399104 deprecation_wrapper.py:119] From /content/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0904 03:39:00.669187 140429746399104 deprecation_wrapper.py:119] From /content/gpt-2/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0904 03:39:00.751445 140429746399104 deprecation_wrapper.py:119] From ./train.py:88: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0904 03:39:00.751766 140429746399104 deprecation_wrapper.py:119] From ./train.py:91: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-09-04 03:39:00.769537: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-09-04 03:39:00.771371: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f65100 executing computations on platform Host. Devices:
2019-09-04 0

Save our checkpoints to start training again later

In [ ]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

Load your trained model for use in sampling below (117M or 345M)

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

W0904 00:38:59.198956 140564883507072 deprecation_wrapper.py:119] From /content/gpt-2/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0904 00:38:59.291688 140564883507072 deprecation_wrapper.py:119] From src/interactive_conditional_samples.py:55: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-09-04 00:38:59.293412: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-04 00:38:59.312397: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-04 00:38:59.313183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-09-04 00:38:59.313431: I tensorflow/stream_ex

To check flag descriptions, use:

In [ ]:
!python3 src/interactive_conditional_samples.py -- --help

W0903 15:49:18.644938 139822344439680 deprecation_wrapper.py:119] From /content/gpt-2/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

NAME
    interactive_conditional_samples.py - Interactively run the model :model_name=117M : String, which model to use :seed=None : Integer seed for random number generators, fix seed to reproduce results :nsamples=1 : Number of samples to return total :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples. :length=None : Number of tokens in generated text, if None (default), is determined by model hyperparameters :temperature=1 : Float value controlling randomness in boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions. :top_k=0 : Integer value controlling diversity. 1 means only 1 word is considered 

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

W0904 00:39:56.692107 140067729090432 deprecation_wrapper.py:119] From /content/gpt-2/gpt-2/src/model.py:147: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0904 00:39:56.783232 140067729090432 deprecation_wrapper.py:119] From src/generate_unconditional_samples.py:52: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-09-04 00:39:56.784858: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-04 00:39:56.802736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-09-04 00:39:56.803459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-09-04 00:39:56.803737: I tensorflow/stream_exe

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help